In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
import numpy as np
import pandas as pd

In [3]:
temp_df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [4]:
df = temp_df.iloc[:20000]
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.review[1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [6]:
df.sentiment.value_counts()

sentiment
negative    10097
positive     9903
Name: count, dtype: int64

In [7]:
df.isna().sum()

review       0
sentiment    0
dtype: int64

In [8]:
df.duplicated().sum()

74

In [9]:
df.drop_duplicates(inplace=True)

/tmp/ipykernel_13/3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [10]:
# basic preprocessing 
# remove tags
# lower case
# remove stopwords

In [11]:
df.review[1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [12]:
import re
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

def clean_text(text):
    text = text.lower()                          # Lowercase the text
    text = re.sub(r'[^\w\s]', '', text)          # Remove punctuation
    words = text.split()
    words = [word for word in words if word not in sw_list]  # Remove stopwords
    return ' '.join(words)


In [13]:
df.review = df.review.apply(clean_text)

/tmp/ipykernel_13/2073461745.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.review = df.review.apply(clean_text)


In [14]:
df.review[1]

'wonderful little production br br filming technique unassuming oldtimebbc fashion gives comforting sometimes discomforting sense realism entire piece br br actors extremely well chosen michael sheen got polari voices pat truly see seamless editing guided references williams diary entries well worth watching terrificly written performed piece masterful production one great masters comedy life br br realism really comes home little things fantasy guard rather use traditional dream techniques remains solid disappears plays knowledge senses particularly scenes concerning orton halliwell sets particularly flat halliwells murals decorating every surface terribly well done'

In [15]:
X = df.iloc[:,0:1]
y = df.sentiment

In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [18]:
y_train = le.fit_transform(y_train)
y_test  = le.transform(y_test)

In [19]:
X_train.shape,y_test.shape

((15940, 1), (3986,))

In [20]:
# Aplying BOW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [21]:
X_train_bow = cv.fit_transform(X_train.review).toarray()
X_test_bow = cv.transform(X_test.review).toarray()

In [22]:
X_train_bow.shape,X_test_bow.shape

((15940, 95964), (3986, 95964))

In [23]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_bow,y_train)

GaussianNB()

In [24]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.654039136979428

Deep Learning Word2Vec approch

In [25]:
import gensim
import nltk
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

In [26]:
story = []
for doc in df.review:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [27]:
model = gensim.models.Word2Vec(
    window = 10,
    min_count = 2
)

In [28]:
model.build_vocab(story)

In [29]:
model.train(story,total_examples = model.corpus_count, epochs = model.epochs)

(11245559, 12088370)

In [30]:
len(model.wv.index_to_key)

50633

In [31]:
def doc_vec_mean(doc):
    sel_doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[sel_doc],axis=0)

In [32]:
doc_vec_mean(df.review.values[0])

array([-0.04904234,  0.28739426, -0.05715181, -0.05029223, -0.12523521,
       -0.21274759,  0.23765929,  0.48283026, -0.27481642, -0.31787485,
       -0.16688134, -0.23271343,  0.00693688, -0.16675086,  0.09290869,
       -0.03995353,  0.19426419, -0.4723204 , -0.02759389, -0.8464609 ,
        0.61313814,  0.01446403,  0.20907559, -0.32705036,  0.11175052,
        0.14088032, -0.239317  ,  0.15493003, -0.13953942,  0.03145045,
        0.2490843 ,  0.23398478,  0.27929863, -0.38429663, -0.5450526 ,
        0.4686652 ,  0.25715736, -0.28633884, -0.6072729 , -0.29174998,
       -0.01104889, -0.2525753 ,  0.00567628,  0.21638802,  0.03119891,
       -0.22662576, -0.44119835, -0.2833529 ,  0.03170708,  0.3943665 ,
        0.15863025, -0.39522108, -0.14318177,  0.0172774 , -0.3174193 ,
        0.03701071,  0.18898591, -0.13338225, -0.7069913 ,  0.05894379,
        0.04637599, -0.1255442 ,  0.24899237, -0.24140279, -0.30929425,
        0.145621  ,  0.04668551,  0.6603996 , -0.65335613,  0.27

In [33]:
from tqdm import tqdm

In [34]:
x = []
for doc in tqdm(df.review.values):
    x.append(doc_vec_mean(doc))

100%|██████████| 19926/19926 [05:33<00:00, 59.66it/s]


In [35]:
X = np.array(x)

In [36]:
X[0]

array([-0.04904234,  0.28739426, -0.05715181, -0.05029223, -0.12523521,
       -0.21274759,  0.23765929,  0.48283026, -0.27481642, -0.31787485,
       -0.16688134, -0.23271343,  0.00693688, -0.16675086,  0.09290869,
       -0.03995353,  0.19426419, -0.4723204 , -0.02759389, -0.8464609 ,
        0.61313814,  0.01446403,  0.20907559, -0.32705036,  0.11175052,
        0.14088032, -0.239317  ,  0.15493003, -0.13953942,  0.03145045,
        0.2490843 ,  0.23398478,  0.27929863, -0.38429663, -0.5450526 ,
        0.4686652 ,  0.25715736, -0.28633884, -0.6072729 , -0.29174998,
       -0.01104889, -0.2525753 ,  0.00567628,  0.21638802,  0.03119891,
       -0.22662576, -0.44119835, -0.2833529 ,  0.03170708,  0.3943665 ,
        0.15863025, -0.39522108, -0.14318177,  0.0172774 , -0.3174193 ,
        0.03701071,  0.18898591, -0.13338225, -0.7069913 ,  0.05894379,
        0.04637599, -0.1255442 ,  0.24899237, -0.24140279, -0.30929425,
        0.145621  ,  0.04668551,  0.6603996 , -0.65335613,  0.27

In [37]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [38]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [39]:
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [40]:
GNB = GaussianNB()
GNB.fit(X_train,y_train)
y_pred = GNB.predict(X_test)
accuracy_score(y_test,y_pred)

0.7669342699448068

In [41]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred2 = rf.predict(X_test)
accuracy_score(y_test,y_pred2)

0.8258906171600602

In [42]:
from sklearn.model_selection import GridSearchCV

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20]
}

gcv = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=params,
    cv=5,                # ✅ 5-fold cross-validation
    verbose=2
)

gcv.fit(X_train, y_train)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=  10.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=  10.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=  10.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=  10.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=  10.7s
[CV] END .....................max_depth=10, n_estimators=200; total time=  21.5s
[CV] END .....................max_depth=10, n_estimators=200; total time=  21.5s
[CV] END .....................max_depth=10, n_estimators=200; total time=  21.4s
[CV] END .....................max_depth=10, n_estimators=200; total time=  21.5s
[CV] END .....................max_depth=10, n_estimators=200; total time=  21.5s
[CV] END .....................max_depth=20, n_estimators=100; total time=  13.6s
[CV] END .....................max_depth=20, n_est

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 20], 'n_estimators': [100, 200]},
             verbose=2)

In [44]:
gcv.best_params_

{'max_depth': 20, 'n_estimators': 200}

In [45]:
rf2 = RandomForestClassifier(max_depth= 10, n_estimators= 100)
rf2.fit(X_train,y_train)
y_pred3 = rf2.predict(X_test)
accuracy_score(y_pred3,y_test)

0.8291520321123934